In [7]:
import asyncio
import os
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from nemoguardrails import LLMRails, RailsConfig
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_cohere import ChatCohere
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import Pinecone as LangchainPinecone
from nemoguardrails.integrations.langchain.runnable_rails import RunnableRails

from nemoguardrails.actions import action
import json
from utils.util import (load_environment_variables,
                        initialize_clients,
                        create_embeddings,
                        create_pinecone_collection)

In [8]:

def get_file_path(filename):
    script_dir = os.path.dirname(os.path.abspath(__file__))
    return os.path.join(script_dir, filename)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


In [9]:
from typing import List

from langchain_cohere import CohereEmbeddings
from nemoguardrails import LLMRails
from nemoguardrails.embeddings.basic import EmbeddingModel
from dotenv import load_dotenv
import os

load_dotenv()

class CohereEmbeddingModel(EmbeddingModel):
    """Cohere embedding model."""

    engine_name = "cohere"

    #embedding_model="embed-english-v3.0", api_key=env_vars["cohere_api_key"]
    def __init__(self, embedding_model: str = "embed-english-v3.0"):
        """Initialize the Cohere embedding model."""
        #super().__init__()
        print("initating embedding model")
        self.cohere_embeddings = CohereEmbeddings(
            model=embedding_model,
            cohere_api_key=os.getenv("COHERE_API_KEY")
        )

    async def encode_async(self, documents: List[str]) -> List[List[float]]:
        """Encode the given documents asynchronously."""
        return self.cohere_embeddings.embed_documents(documents)

    def encode(self, documents: List[str]) -> List[List[float]]:
        """Encode the given documents."""
        return self.cohere_embeddings.embed_documents(documents)

def init(app: LLMRails):
    print("init LLMRails model")
    app.register_embedding_provider(CohereEmbeddingModel, "cohere")


In [10]:
colang_content = """
define user greeting
  "Hi"
  "Hello,how are you"
  "Can you help me"

define bot greeting
  "Hello,how may I assist you today"
  "Hello"

define user ask politics
  "what do you think about the new president?"
  "who do you support in current elections"
  "what are your political beliefs?"
  "thoughts on the president?"
  "left wing"
  "right wing"

define bot answer politics
  "I am sorry,I only answer questions on deep learning, I do not make political statements"

define flow politics
  user ask politics
  bot answer politics
  bot offer_help

define user ask deeplearning
  "what is deep learning"
  "what is a CNN"
  "what is a RNN"
  "what is CML"
  "what is model training"

define flow deeplearning
  bot greeting
  user ask deeplearning
  $answer = execute qa_chain(query=$last_user_message)
  bot $answer


define user ask programming
  "write a python code"
  "can you code"
  "programming question"
  "write a python code ?"

define bot answer programming
  "While I can discuss programming concepts related to deep learning, I'm not designed to write or execute code. If you have questions about programming in the context of deep learning, I'd be happy to help with that. What specific aspect of programming in deep learning would you like to know about?"

define flow programming
  user ask programming
  bot answer programming
  bot offer_help

"""
yaml_content = """
models:
 - type: main
   engine: cohere
   model: command-r-plus
 - type: embeddings
   engine: cohere
   model: embed-english-v3.0
"""

In [11]:

env_vars = load_environment_variables()
index_name = env_vars["pinecone_index"]
cohere_client, pinecone_client = initialize_clients(env_vars["cohere_api_key"], env_vars["pinecone_api_key"])
embeddings = create_embeddings(env_vars["cohere_api_key"])
index = create_pinecone_collection(pinecone_client, index_name)
model = ChatCohere(model="command-r-plus", temperature=0)

rails_config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)

rag_rails = LLMRails(rails_config, verbose=False)
rag_rails.register_embedding_provider(CohereEmbeddingModel, "cohere")

rag_template = PromptTemplate.from_template(
        """
        You are a helpful assistant. Use the following pieces of context to answer the question at the end. 
        Answer always start with [Naga:]. If you don't know the answer, just say that I am sorry. 
        You are a helpful assistant offering further help. 
        Don't try to make up an answer.

        Context: {context}

        Question: {question}

        Answer:"""
)

docsearch = LangchainPinecone.from_existing_index(index_name=index_name, embedding=embeddings)

def create_documents(input_dict):
    question = input_dict["question"]
    docs = docsearch.similarity_search(question)
    context = format_docs(docs)
    return {"context": context, "question": question}

def print_full_prompt(prompt):
    print("Full Generated Prompt:")
    print("----------------------")
    print(prompt)
    print("----------------------")
    return prompt

rag_chain = (
            RunnablePassthrough()
            | create_documents
            | rag_template
            | print_full_prompt  # Add this step to print the full prompt
            | model
            | StrOutputParser()
)

rag_rails.register_action(rag_chain, name="qa_chain")

print("done")


2024-10-05 19:30:52 - util.py - Creating 1024-dimensional index called 'ml-docs'...
2024-10-05 19:30:52 - util.py - Index already exists, continuing...
2024-10-05 19:30:53 - util.py - Checking Pinecone for active indexes...
2024-10-05 19:30:53 - util.py - Getting description for 'ml-docs'...
2024-10-05 19:30:53 - util.py - Getting 'ml-docs' as object...
2024-10-05 19:30:53 - util.py - Success


Active indexes: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'ml-docs-04fgi1r.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'ml-docs',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'customer-reviews-1-04fgi1r.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'customer-reviews-1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}
Description: {'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'ml-docs-04fgi1r.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'ml-docs',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'stat

ValueError: cohere already exists in the registry

In [12]:
await rag_rails.generate_async(prompt="Write about CML ?")

"I'd be happy to help!\nContinuous Integration and Continuous Deployment (CI/CD) are essential practices in modern software development, and they play a crucial role in Machine Learning (ML) and Deep Learning (DL) as well. CI/CD enables developers to automate the testing, integration, and deployment of their code changes, ensuring that the ML/DL models are trained and deployed efficiently and reliably.\nOne key benefit of CI/CD in the context of ML/DL is the ability to catch issues early in the development cycle. By automating the testing and validation process, developers can identify and fix bugs or incompatibilities before they cause major problems. This is especially important in ML/DL projects, where small changes in the code or data can have significant impacts on the model's performance.\nAnother advantage of CI/CD is the ability to deploy models quickly and consistently. With automated deployment pipelines, developers can easily roll out updates to production environments, ensu